In [158]:
import pandas as pd

In [159]:
poleval_df = pd.read_csv("poleval2019_task2_training_190221/index.tsv", sep="\t", header=None)
poleval_df.columns = ["phrase_id", "doc_id", "text", "lemma"]
poleval_df.head()

,phrase_id,doc_id,text,lemma
0,31822,99883,Toronto Dominion Centre,Toronto Dominion Centre
1,40025,99883,Toronto,Toronto
2,343873,99883,kompleks handlowo-kulturalny,kompleks handlowo-kulturalny
3,31833,99883,Joe Fafard,Joe Fafard
4,327365,99883,kanadyjskim,kanadyjski


In [160]:
poleval_df.shape

(22177, 4)

In [161]:
poleval_df = poleval_df[poleval_df["text"] != poleval_df["lemma"]]

In [162]:
poleval_df.shape

(10720, 4)

In [163]:
mwe_df = poleval_df[poleval_df["text"].str.split().str.len() > 1]
mwe_df.head()

,phrase_id,doc_id,text,lemma
5,343872,99883,Ludwiga Mies van der Rohe,Ludwig Mies van der Rohe
18,343879,100499,samolotu PZL.23 „Karaś”,samolot PZL.23 „Karaś”
19,31843,100499,Ministerstwa Komunikacji,Ministerstwo Komunikacji
22,343880,100499,silnikiem Pratt-Whitney Wasp,silnik Pratt-Whitney Wasp
28,31839,100499,Stanisława Praussa,Stanisław Prauss


In [164]:
mwe_df.shape

(4783, 4)

In [165]:
mwe_df[['text', 'lemma']].sample(10)

,text,lemma
14383,Specjalnego Ośrodka Szkolno-Wychowawczego,Specjalny Ośrodek Szkolno-Wychowawczy
14373,Zespołu Szkół Ogólnokształcących,Zespół Szkół Ogólnokształcących
16352,Best Malla,Best Mall
10215,Morza Bałtyckiego,Morze Bałtyckie
15883,Ministra Obrony Narodowej,Minister Obrony Narodowej
4334,Pucharu Polski,Puchar Polski
12583,Rady Miasta,Rada Miasta
3998,Gienricha Jagody,Gienricha Jagoda
16116,Placu Zbawiciela,Plac Zbawiciela
19567,Fundacji Wikimedia,Fundacja Wikimedia


In [166]:
mwe_df.isna().sum()

phrase_id    0
doc_id       0
text         0
lemma        1
dtype: int64

In [167]:
mwe_df[mwe_df["lemma"].isna()]

,phrase_id,doc_id,text,lemma
3332,307628,101286,San Diego Rockets,NaN


In [168]:
mwe_df = mwe_df.dropna()

In [170]:
# # For every word in the text, I check if it is the same as in the lemma.
# # If it is not, I check form of the word using Morfeusz2.

from morfeusz2 import Morfeusz
morfeusz = Morfeusz()  

def get_case_and_number(phrase, lemma):
    """
    Function to get case and number of a word using Morfeusz2.
    :param phrase: str, phrase to be analyzed
    :return: str, number and case of the word
    """
    words = phrase.split()
    lemmas = lemma.split() if lemma else []
    for i, word in enumerate(words):

        if i < len(lemmas) and word == lemma.split()[i]:
            continue

        analyses = morfeusz.analyse(word)

        for _, _, interpretations in analyses:
            tag = interpretations[2].split(":")
            if tag[0] in ["subst", "depr"]:
                return tag[1], tag[2] #number, case



In [171]:
get_case_and_number("Wizowego Systemu Informacyjnego", "Wizowy System Informacyjny")

('sg', 'gen')

In [172]:
mwe_df[["number", "case"]] = mwe_df.apply(lambda row: pd.Series(get_case_and_number(row["text"], row["lemma"])), axis=1)

In [173]:
mwe_df.sample(10)

,phrase_id,doc_id,text,lemma,number,case
15282,288725,102479,Unii Europejskiej,Unia Europejska,sg,gen
21599,490565,107342,prezydenta Obamy,prezydent Obama,sg,gen.acc
5203,56569,101406,12 pułku huzarów,12 pułk huzarów,sg,gen
14229,338099,102382,Pucharze Polski,Puchar Polski,sg,loc
14412,484878,102386,Harry’ego Pottera,Harry Potter,sg,gen.acc
12608,466308,102160,Gminy Lublin,Gmina Lublin,pl,nom.acc.voc
7706,63446,101599,Robertem Tonnorem,Robert Tonnor,sg,inst
5173,56534,101405,Osiedle XXV-lecia PRL,osiedle XXV-lecia PRL,sg,nom.acc.voc
11933,310513,102084,"II Regionalnego Konkursu Fotograficznego "" Ozo...","II Regionalny Konkurs Fotograficzny "" Ozorków ...",sg,gen
7549,54523,101590,Szkole Weterynaryjnej,Szkoła Weterynaryjna,sg,dat.loc


In [174]:
mwe_df.isna().sum()

phrase_id      0
doc_id         0
text           0
lemma          0
number       300
case         300
dtype: int64

In [175]:
mwe_df[mwe_df["number"].isna()]

,phrase_id,doc_id,text,lemma,number,case
313,32161,100521,Bezkresnego Imperium,Bezkresne Imperium,NaN,NaN
327,32157,100521,Bezkresnego Imperium,Bezkresne Imperium,NaN,NaN
401,99725,100524,Dolce & Gabbana,Dolce &amp; Gabbana,NaN,NaN
423,344047,100525,Johanna Mühlegga,Johann Mühlegga,NaN,NaN
766,41316,100608,Interdyscyplinarnego Centrum Modelowania Matem...,Interdyscyplinarne Centrum Modelowania Matemat...,NaN,NaN
...,...,...,...,...,...,...
21311,490467,107326,Nicolasem Sarkozy,Nicolas Sarkozy,NaN,NaN
21369,326365,107329,Andry'ego Rajoeliny,Andry Rajoelina,NaN,NaN
21462,326487,107334,Harrim Ollim,Harri Olli,NaN,NaN
21869,328869,107355,Powiatowego Centrum Zarządzania Kryzysowego w ...,Powiatowe Centrum Zarządzania Kryzysowego w St...,NaN,NaN


In [177]:
len(mwe_df["lemma"].unique())

3649